In [155]:
import os
from dotenv import load_dotenv

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableBranch, RunnableLambda, RunnableParallel
from langchain.output_parsers import PydanticOutputParser
from langchain_core.output_parsers import StrOutputParser
from pydantic import BaseModel, Field
from langchain_groq import ChatGroq
from langchain_anthropic import ChatAnthropic


import re
from datetime import datetime

In [156]:
policy = '''
Return Window
--------------------------------
Most items sold on meesho are eligible for return or replacement within 7–10 days of delivery.

Certain product categories (e.g., electronics, large appliances) may have different return timelines. The applicable window will be clearly mentioned on the product detail page.

Conditions for Returns
---------------------------------
To qualify for a return or replacement:

The item must be in its original condition, including brand/manufacturer packaging, original box, MRP tags, user manual, warranty cards, and all accessories.

It must be unused, unwashed, and undamaged.

Items should not be tampered with or altered in any form.

For products requiring installation, return requests will only be accepted after a technician visit.

Acceptable Reasons for Return
------------------------------------
Returns or replacements will be accepted under the following conditions:

1. Damaged or Defective Item
Product is broken, dented, scratched, or physically damaged.

Product is non-functional or shows a manufacturing defect.

2. Wrong Item Delivered
You received a different item, size, color, or model than what you ordered.

3. Item Missing Components
Accessories such as chargers, cables, or manuals are missing.

4. Product Not as Described
The product received is significantly different from what was shown on the product page (in design, features, specs, etc.).

5. Item Expired
Products like food, cosmetics, or medicines that arrive after expiry date.

Note: Valid photographic or video evidence may be required for claims involving physical damage or missing components.

Non-Eligible Returns
--------------------------------------------
Your return request will be rejected in the following scenarios:
- Return request raised after the return window.
- Product shows signs of use or misuse (stains, scratches, broken seals).
- Item returned is not the same as delivered (wrong item returned).

Requests made for buyer’s remorse will be rejected for refund:
- “I changed my mind”
- “Item doesn’t suit me”
- "Not what I expected”
- Packaging-only damage (if the product itself is functional and undamaged).

For hygiene and safety reasons, returns are not accepted for:
- Innerwear, lingerie, socks
- Personal grooming items
- Baby care and sanitary products
- Food and beverages
- Products marked as non-returnable on the product page (e.g., digital goods, software licenses).

Category-Specific Return Guidelines
---------------------------------------------------------------
1) Electronics and Appliances
Returns only accepted if the issue is reported within 7 days from delivered date and is verified by an authorized technician.
Damage claims must be supported with installation reports or inspection notes.
Mobile phones, laptops, and accessories must be returned with screen locks disabled and factory reset.

2) Apparel & Footwear
Items must be tried on without removing tags and must be returned in sellable condition.
Footwear must be returned in the original box with no wear or dirt on soles.

3) Furniture & Large Items
Returns will only be processed after on-site inspection.
For self-assembled items, returns will not be accepted for damage due to incorrect assembly by the customer.

4) Beauty & Personal Care
Items once opened or used cannot be returned.
Only defective or damaged items (reported on day of delivery) will be considered.

5) Replacements
Replacements are subject to product availability. If the same item is not available, you may opt for a full refund or a different item (as per platform policy).

'''

orders = {
  "ORD12345": {
    "status": "Delayed",
    "eta": "2025-07-16",
    "courier_notes": "Shipment scheduled on 10th but stuck in Pune hub. Flooding bad. Road closed 2 days. Team try delivery on 11th but way blocked. Will try again when road opens, weather still bad."
  },
  "ORD12346": {
    "status": "Delayed",
    "eta": "2025-07-14",
    "courier_notes": "Package was ready on time but rider couldn’t pick up due to petrol strike in city. Station near warehouse not operating. Driver told to come next morning. Hoping to move tomorrow."
  },
  "ORD12347": {
    "status": "Delayed",
    "eta": "2025-07-15",
    "courier_notes": "Late dispatch. First try missed because customer gate was locked. Neighbor said person not home. Will attempt again tomorrow maybe early morning if weather okay."
  },
  "ORD12348": {
    "status": "Shipped",
    "eta": "2025-07-13",
    "courier_notes": "Shipment collected after delay due to barcode not scanning. Tried twice. Scanner issue fixed around 4pm. Package now moved from hub, in transit now."
  },
  "ORD12349": {
    "status": "Delayed",
    "eta": "2025-07-18",
    "courier_notes": "Courier boy fell sick. Replacement rider was not available immediately. Package still in warehouse, manager says will send with next batch if rider free on Monday."
  },
  "ORD12350": {
    "status": "Delayed",
    "eta": "2025-07-17",
    "courier_notes": "Package was marked ready but got mixed in wrong bag going to different city. Was sent to Nagpur instead of Mumbai. Returned today. Will reroute and dispatch again by evening shift."
  },
  "ORD12351": {
    "status": "Out for delivery",
    "eta": "2025-07-12",
    "courier_notes": "Item with rider since morning. Traffic in city very heavy today due to local festival. Will try to cover this address by last slot around 6pm–9pm."
  },
  "ORD12352": {
    "status": "Delayed",
    "eta": "2025-07-15",
    "courier_notes": "Package not moved. System error — AWB not generated in dispatch tool. Spoke with ops team, reprinting label now. Rider waiting, will go once scan shows green."
  },
  "ORD12353": {
    "status": "Shipped",
    "eta": "2025-07-14",
    "courier_notes": "Sent out with batch today. Rider mentioned bike breakdown halfway. Package returned to hub and re-assigned to second rider by 4:30pm. Now back on route."
  },
  "ORD12354": {
    "status": "Delayed",
    "eta": "2025-07-19",
    "courier_notes": "Customer asked for delivery after 5pm but rider reached at 2pm. No one opened. Address confirmed correct. Will mark for evening slot re-attempt."
  }
}


### Branching decision

In [157]:
class RoutingIntent(BaseModel):
    intent: str = Field(
        description='The classified user intent. It should be one of the following: "return_refund", "track_order", or "other".'
    )
    user_question: str = Field(description="The input query user has")
    order_id: str = Field(description="order id given by user")


parser_branch = PydanticOutputParser(pydantic_object=RoutingIntent)
format_instructions_branch = parser_branch.get_format_instructions()

In [158]:
prompt_branch = ChatPromptTemplate.from_template('''You are a support query classifier for an e-commerce platform.

Your job is to read a customer's message and decide which type of support workflow should be triggered.

Think through the problem step by step:
1. Read and understand the user's message.
2. Identify any keywords or phrases that indicate what the user wants.
3. Decide which category the query falls under:
   - If the user is reporting a damaged, defective, incorrect, or unwanted item and is asking for a return, replacement, or refund → it's a **Return or Refund** case.
   - If the user is asking for delivery updates, shipment status, or estimated arrival → it's an **Order Tracking** case.
   - If it doesn't fall into either category, it's something else.

Use one of the following intents:
- "return_refund"
- "track_order"
- "other"
                                                 
**Also give input user question and order id as output as it is along with intent


Now classify the following user message:
{user_query}
                                                 
order_id:
{order_id}
                                        
{format_instructions_branch}
                                              
                                              ''')


llama ="llama-3.1-8b-instant"

model_branch = ChatGroq(temperature=0, model_name=llama)

branch_chain = prompt_branch | model_branch | parser_branch

### Refund Chain

In [159]:
class RefundEvaluation(BaseModel):
    category_of_item: str = Field(description="Category of the item (e.g., electronics, apparel, personal care)")
    item_name: str = Field(description="The name or type of the item mentioned in the return request")
    qualifies_for_refund: str = Field(description='Either "Yes" or "No" indicating whether the item qualifies for refund')
    reason_for_qualification: str = Field(description="Brief justification for the refund decision based on the return policy")


parser_refund = PydanticOutputParser(pydantic_object=RefundEvaluation)
format_instructions_refund = parser_refund.get_format_instructions()

In [160]:
prompt_refund = ChatPromptTemplate.from_template('''You are a return and refund evaluator for an e-commerce platform.

Below is our return & refund policy:
---
{policy_text}
---

A customer has submitted the following return request:
{user_query}

### Task:

Think step by step:
1. **Identify the item** being referred to and the problem described by the customer.
2. **Classify the product** into a clear category (e.g., electronics, apparel, personal care).
3. **Strictly verify eligibility based on policy**:
   - For the specific category, check from policy text what are the conditions that will lead to reject refund
   - Match those conditions with the reasons mentioned by user.
   - If any condition mentioned by user will lead to rejection of refund then reject the claim else pass the claim
                                                  
{format_instructions_refund}

                                              
                                              ''')


#model_refund = ChatAnthropic(temperature=0, model_name='claude-3-5-sonnet-20240620')
llama ='llama3-70b-8192'

model_refund = ChatGroq(temperature=0, model_name=llama)


refund_chain = prompt_refund | model_refund | parser_refund

## Track chain

In [184]:
def get_order_details(user_prompt):

    question = user_prompt['user_query']
    order_id = user_prompt['order_id']

    order_status = orders[order_id]

    return {"input" : order_status}

track_chain = RunnableLambda(get_order_details)

## Map Chain 

In [185]:
def map_details(input):

    if input.intent == 'return_refund':
        question = input.user_question
        return {
            "intent": input.intent, 
            "user_query": question,
            "policy_text": policy,
            "format_instructions_refund": format_instructions_refund
        }

    else:
        question = input.user_question
        id = input.order_id
        return {
            "intent": input.intent, 
            "user_query": question,
            "order_id": id
        }

map_chain = RunnableLambda(map_details)

### Branch Chain

In [186]:
general_chain = RunnablePassthrough()

branch = RunnableBranch(
    (lambda x: "return_refund" in x['intent'].lower(), refund_chain),
    (lambda x: "track_order" in x['intent'].lower(), track_chain),
    general_chain,
)

## Summarize chain

In [187]:
prompt = ChatPromptTemplate.from_template('''Please summarize data mentioned below and repond to the customer as friendly customer service agent.
                                          input:
                                          {input}
                                              ''')

llama ="llama-3.1-8b-instant"

model = ChatGroq(temperature=0, model_name=llama)

output_parser_str = StrOutputParser()

chain_final = prompt | model | output_parser_str

# Final chain

In [188]:
full_chain = branch_chain | map_chain | branch | chain_final

In [ ]:
q = '''
Hi, I bought a wireless earbud last week from your platform. It worked fine initially, but now the right earbud isn’t charging
I've already thrown away the box but still have the invoice. Let me know if i can get my money back
'''

q = '''
Hey, I ordered a gaming mouse 5 days ago and it still shows 'Processing' on the app. Can you check what's going on and when it will be delivered?
'''

order = 'ORD12345'

response_branch = full_chain.invoke({"user_query":q, "order_id": order,  "format_instructions_branch":format_instructions_branch})

In [192]:

print(response_branch)

Thank you for reaching out to us about your Wireless Earbud. I'd be happy to help you with your concern.

I've taken a look at your order and I see that it falls under the Electronics category. Unfortunately, you've experienced a manufacturing defect with your right earbud, which is not charging. I completely understand how frustrating that must be for you.

The good news is that you qualify for a refund because you reported the issue within the return window and it's an acceptable reason for return. Our return policy is designed to ensure that you're satisfied with your purchase, and I'm happy to assist you with the refund process.

To proceed, I just need to confirm a few details with you. Can you please confirm your order number and the shipping address you'd like the refund to be sent to? I'll also need to provide you with a return shipping label so you can send the Wireless Earbud back to us.

Thank you for your patience and understanding. I'm here to help and I'll do my best to g